In [1]:
import glob
import numpy as np
from numpy import asarray
import os
from os import listdir
import cv2

In [5]:
a_img = []
for img in glob.glob( "images_sub11/a_image/*.png"):
    if(len(a_img)==3000):
        break
    n= cv2.imread(img)
    a_img.append(n/255.0)

In [6]:
len(a_img)

3000

In [7]:
i_img = []
f=0
for img in glob.glob("images_sub11/i_image/*.png"):
    if(len(i_img)==3000):
        break
    n= cv2.imread(img)
    i_img.append(n/255.0)
    f+=1

In [20]:
len(u_img)

3000

In [8]:
u_img = []
for img in glob.glob("images_sub11/u_image/*.png"):
    if(len(u_img)==3000):
        break
    n= cv2.imread(img)
    u_img.append(n/255.0)

In [9]:
labels =np.zeros(3000*3)
labels[0:3000]=1
labels[3000:2*3000]=2

In [10]:
data=[]
for i in a_img:
  ii = cv2.resize(i, (128,128))
  data.append(ii)

In [11]:
for i in i_img:
  ii = cv2.resize(i, (128,128))
  data.append(ii)

In [12]:
for i in u_img:
  ii = cv2.resize(i, (128,128))
  data.append(ii)

In [25]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.20, random_state=42)

In [26]:
X_train = np.array(X_train, dtype=np.float32)
X_test = np.array(X_test, dtype=np.float32)

In [27]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras import optimizers

In [28]:
y_train = to_categorical(y_train, 3)
y_test = to_categorical(y_test, 3)

In [29]:
len(X_train)

7200

In [31]:
batch_size = 16
nb_classes =3
nb_epochs = 10
img_rows, img_columns = 128, 128
img_channel = 3
nb_filters = 32
nb_pool = 2
nb_conv = 3

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation=tf.nn.relu,
                           input_shape=(128, 128, 3)),
    tf.keras.layers.MaxPooling2D((2, 2), strides=2),
    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D((2, 2), strides=2),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(3,  activation=tf.nn.softmax)
])
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(X_train, y_train, batch_size = batch_size, epochs = nb_epochs, verbose = 1, validation_data = (X_test, y_test))


Epoch 1/10
450/450 [==============================] - 77s 166ms/step - loss: 1.1093 - accuracy: 0.3424 - val_loss: 1.0632 - val_accuracy: 0.4567
Epoch 2/10
450/450 [==============================] - 74s 165ms/step - loss: 0.8814 - accuracy: 0.5857 - val_loss: 0.5063 - val_accuracy: 0.8228
Epoch 3/10
450/450 [==============================] - 74s 163ms/step - loss: 0.4005 - accuracy: 0.8451 - val_loss: 0.2369 - val_accuracy: 0.9217
Epoch 4/10
450/450 [==============================] - 74s 163ms/step - loss: 0.1774 - accuracy: 0.9376 - val_loss: 0.1333 - val_accuracy: 0.9561
Epoch 5/10
450/450 [==============================] - 74s 165ms/step - loss: 0.0913 - accuracy: 0.9679 - val_loss: 0.1336 - val_accuracy: 0.9522
Epoch 6/10
450/450 [==============================] - 73s 163ms/step - loss: 0.0626 - accuracy: 0.9786 - val_loss: 0.1076 - val_accuracy: 0.9656
Epoch 7/10
450/450 [==============================] - 74s 164ms/step - loss: 0.0443 - accuracy: 0.9854 - val_loss: 0.1134 - val_ac

In [32]:
score = model.evaluate(X_test, y_test)
print("Test accuracy: ", score[1])

57/57 [==============================] - 4s 68ms/step - loss: 0.0979 - accuracy: 0.9683
Test accuracy:  0.9683333039283752


In [ ]:
# TRANSFORMER

In [ ]:
num_classes = 3
input_shape = (128, 128, 3)


In [ ]:
learning_rate = 0.001
weight_decay = 0.0001
batch_size = 200
num_epochs = 100
image_size = 128  # We'll resize input images to this size
patch_size = 6  # Size of the patches to be extract from the input images
num_patches = (image_size // patch_size) ** 2
projection_dim = 64
num_heads = 4
transformer_units = [
    projection_dim * 2,
    projection_dim,
]  # Size of the transformer layers
transformer_layers = 8
mlp_head_units = [2048, 1024]

In [ ]:
data_augmentation = keras.Sequential(
    [
        layers.Normalization(),
        layers.Resizing(image_size, image_size),
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(factor=0.02),
        layers.RandomZoom(
            height_factor=0.2, width_factor=0.2
        ),
    ],
    name="data_augmentation",
)
# Compute the mean and the variance of the training data for normalization.
data_augmentation.layers[0].adapt(x_train)


In [ ]:
def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x

In [ ]:
class Patches(layers.Layer):
    def __init__(self, patch_size):
        super().__init__()
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(4, 4))
image = x_train[np.random.choice(range(x_train.shape[0]))]
plt.imshow(image.astype("uint8"))
plt.axis("off")

resized_image = tf.image.resize(
    tf.convert_to_tensor([image]), size=(image_size, image_size)
)
patches = Patches(patch_size)(resized_image)
print(f"Image size: {image_size} X {image_size}")
print(f"Patch size: {patch_size} X {patch_size}")
print(f"Patches per image: {patches.shape[1]}")
print(f"Elements per patch: {patches.shape[-1]}")

n = int(np.sqrt(patches.shape[1]))
plt.figure(figsize=(4, 4))
for i, patch in enumerate(patches[0]):
    ax = plt.subplot(n, n, i + 1)
    patch_img = tf.reshape(patch, (patch_size, patch_size, 3))
    plt.imshow(patch_img.numpy().astype("uint8"))
    plt.axis("off")

In [ ]:
class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super().__init__()
        self.num_patches = num_patches
        self.projection = layers.Dense(units=projection_dim)
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded

In [ ]:
def create_vit_classifier():
    inputs = layers.Input(shape=input_shape)
    # Augment data.
    augmented = data_augmentation(inputs)
    # Create patches.
    patches = Patches(patch_size)(augmented)
    # Encode patches.
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)

    # Create multiple layers of the Transformer block.
    for _ in range(transformer_layers):
        # Layer normalization 1.
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        # Create a multi-head attention layer.
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=0.1
        )(x1, x1)
        # Skip connection 1.
        x2 = layers.Add()([attention_output, encoded_patches])
        # Layer normalization 2.
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        # MLP.
        x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.1)
        # Skip connection 2.
        encoded_patches = layers.Add()([x3, x2])

    # Create a [batch_size, projection_dim] tensor.
    representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    representation = layers.Flatten()(representation)
    representation = layers.Dropout(0.5)(representation)
    # Add MLP.
    features = mlp(representation, hidden_units=mlp_head_units, dropout_rate=0.5)
    # Classify outputs.
    logits = layers.Dense(num_classes)(features)
    # Create the Keras model.
    model = keras.Model(inputs=inputs, outputs=logits)
    return model

In [ ]:
def run_experiment(model):
    optimizer = tfa.optimizers.AdamW(
        learning_rate=learning_rate, weight_decay=weight_decay
    )

    model.compile(
        optimizer=optimizer,
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[
            keras.metrics.SparseCategoricalAccuracy(name="accuracy"),
            keras.metrics.SparseTopKCategoricalAccuracy(5, name="top-5-accuracy"),
        ],
    )

    checkpoint_filepath = "/tmp/checkpoint"
    checkpoint_callback = keras.callbacks.ModelCheckpoint(
        checkpoint_filepath,
        monitor="val_accuracy",
        save_best_only=True,
        save_weights_only=True,
    )

    history = model.fit(
        x=x_train,
        y=y_train,
        batch_size=batch_size,
        epochs=num_epochs,
        validation_split=0.1,
        callbacks=[checkpoint_callback],
    )

    model.load_weights(checkpoint_filepath)
    _, accuracy, top_5_accuracy = model.evaluate(x_test, y_test)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")
    print(f"Test top 5 accuracy: {round(top_5_accuracy * 100, 2)}%")

    return history


vit_classifier = create_vit_classifier()
history = run_experiment(vit_classifier)